# Pong-v0 example

This is a detailed tutorial, so if you just want to run the example, go to the top libary directory and run `julia pong_mail.jl` on the command line.

In [1]:
using Gym
using Images
using Colors
include("dqn.jl")
using DQN

I tensorflow/stream_executor/dso_loader.cc:111] successfully opened CUDA library libcublas.so locally
I tensorflow/stream_executor/dso_loader.cc:111] successfully opened CUDA library libcudnn.so.5 locally
I tensorflow/stream_executor/dso_loader.cc:111] successfully opened CUDA library libcufft.so locally
I tensorflow/stream_executor/dso_loader.cc:111] successfully opened CUDA library libcuda.so.1 locally
I tensorflow/stream_executor/dso_loader.cc:111] successfully opened CUDA library libcurand.so locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUDA library libcublas.so locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUDA library libcudnn.so locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUDA library libcufft.so locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUDA library libcuda.so.1 locally
I tensorflow/stream_executor/dso_loader.cc:128] successfully opened CUDA library lib

## Loading the function for the default network for the Pong-v0 example

This is a default convolutional neural network that follows the same architecture used in the DeepMind Atari paper. The user can define a more complex network architecture as long as the function still returns the required outputs and takes in the required inputs as described in the `README.md`.

In [2]:
include("convnet1.jl") # createNetwork defined here

createNetwork (generic function with 2 methods)

## Setting up the OpenAI gym environment and useful variables

The environment is needed for the `frame_step` function and also needs to get passed into `trainDQN` and `simulateDQN` to get the monitoring capabilities to work.

In [3]:
TIME_FRAMES = 4 # number of time frames in the state

env = GymEnvironment("Pong-v0")
@show ACTIONS = n_actions(env)         # number of valid actions
# @show STATE_SHAPE = obs_dimensions(env)[0]
@show STATE_SHAPE = (80, 80, TIME_FRAMES)

ACTIONS = n_actions(env) = 6
STATE_SHAPE = (80,80,TIME_FRAMES) = (80,80,4)


[2016-12-11 03:08:05,844] Making new env: Pong-v0


(80,80,4)

# Setting up function that interacts with the env

Observe that since the state for Pong-v0 is an image, we need to scale it down, grayscale it and stack them consecutively to so time information is included in each state. The frame step function is a wrapping of the OpenAI Gym step function and normalizes the rewards to be between $[-1,1]$. The frame step function will depend on the environment/model, but if the user is interacting with OpenAI Gym, not very much work is needed here.

In [4]:
function preprocess(x, prev_state)
    grayscale = convert(Image{Gray}, colorim(x))
    resized = Images.imresize(grayscale, (80, 80))
    x = convert(Array{Float32}, data(resized))
    if prev_state == nothing
        cat(3, x, x, x, x)
    else
        cat(3, x, prev_state[:, :, 1:end-1])
    end
end

function frame_step(action, prev_state)
    x_t, r_t, is_terminal = step!(env, action)
    # render(env)
    s_t = preprocess(x_t, prev_state)
    s_0 = is_terminal ? preprocess(reset(env), nothing) : nothing
    s_t, r_t / 200.0, is_terminal, s_0
end

frame_step (generic function with 1 method)

# Initializing the hyperparameters

Look at `dqn.jl` for descriptions of the hyperparameters included in the custom type `HyperParameters`. The fields of are the parameters that the user will need to tune to get better performance on the network.

In [5]:
hyper_params = HyperParameters(ACTIONS, STATE_SHAPE)

DQN.HyperParameters(3.0f-5,0.99f0,1000,4000.0f0,0.05f0,1.0f0,20000,32,1,7500,100000,6,(80,80,4))

# Training the DQN

Running the function below will train the DQN, save the weights/vids/logs and also result in a very long output, which was why max number of episodes is so low. If the user wants to train the DQN, it is best to run `julia pong_main.jl` on the command line.

In [6]:
hyper_params.max_num_episodes = 10
hyper_params.observe = 5
trainDQN(env, frame_step, createNetwork, hyper_params, "test")

[2016-12-11 02:50:02,961] Creating monitor directory test/videos
[2016-12-11 02:50:03,011] Starting new video recorder writing to /home/carol/DQN.jl/test/videos/openaigym.video.0.16329.video000000.mp4
I tensorflow/core/common_runtime/gpu/gpu_device.cc:951] Found device 0 with properties: 
name: GeForce GTX 1080
major: 6 minor: 1 memoryClockRate (GHz) 1.86
pciBusID 0000:03:00.0
Total memory: 7.92GiB
Free memory: 7.45GiB
I tensorflow/core/common_runtime/gpu/gpu_device.cc:972] DMA: 0 
I tensorflow/core/common_runtime/gpu/gpu_device.cc:982] 0:   Y 
I tensorflow/core/common_runtime/gpu/gpu_device.cc:1041] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX 1080, pci bus id: 0000:03:00.0)


starting training
Finished episode     0. Reward=  -0.100


[2016-12-11 02:51:22,549] Starting new video recorder writing to /home/carol/DQN.jl/test/videos/openaigym.video.0.16329.video000001.mp4


Finished episode     1. Reward=  -0.100
Finished episode     2. Reward=  -0.100
Finished episode     3. Reward=  -0.105
Finished episode     4. Reward=  -0.105
Finished episode     5. Reward=  -0.105
Finished episode     6. Reward=  -0.105
Finished episode     7. Reward=  -0.105


[2016-12-11 02:59:48,202] Starting new video recorder writing to /home/carol/DQN.jl/test/videos/openaigym.video.0.16329.video000008.mp4


Finished episode     8. Reward=  -0.105
Finished episode     9. Reward=  -0.100


[2016-12-11 03:02:21,396] Finished writing results. You can upload them to the scoreboard via gym.upload('/home/carol/DQN.jl/test/videos')


# Simulating the DQN from saved weights

The below function loads the pre-trained weights (which have only been trained for 8 hours) and simulates for 2 episodes. Restart the kernel if `trainDQN` was run before this or it will crash, which is another reason why it is best to run `julia pong_main.jl` on the command line.

In [6]:
simulateDQN(env, frame_step, createNetwork, "Pong-v0/saved_wgts/weights-300", 2, hyper_params)

Finished episode     0. Reward=  -0.070


[2016-12-11 03:08:08,856] Creating monitor directory /tmp/dqn/monitor/exp_Pong-v0_2016-12-11T03:08:08.834
[2016-12-11 03:08:08,908] Starting new video recorder writing to /tmp/dqn/monitor/exp_Pong-v0_2016-12-11T03:08:08.834/openaigym.video.0.16939.video000000.mp4
I tensorflow/core/common_runtime/gpu/gpu_device.cc:951] Found device 0 with properties: 
name: GeForce GTX 1080
major: 6 minor: 1 memoryClockRate (GHz) 1.86
pciBusID 0000:03:00.0
Total memory: 7.92GiB
Free memory: 7.45GiB
I tensorflow/core/common_runtime/gpu/gpu_device.cc:972] DMA: 0 
I tensorflow/core/common_runtime/gpu/gpu_device.cc:982] 0:   Y 
I tensorflow/core/common_runtime/gpu/gpu_device.cc:1041] Creating TensorFlow device (/gpu:0) -> (device: 0, name: GeForce GTX 1080, pci bus id: 0000:03:00.0)
[2016-12-11 03:10:37,390] Starting new video recorder writing to /tmp/dqn/monitor/exp_Pong-v0_2016-12-11T03:08:08.834/openaigym.video.0.16939.video000001.mp4


Finished episode     1. Reward=  -0.060


[2016-12-11 03:12:55,514] Finished writing results. You can upload them to the scoreboard via gym.upload('/tmp/dqn/monitor/exp_Pong-v0_2016-12-11T03:08:08.834')
